# Cats Vs Dogs
I'm going to use [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) to finetune a pre-existing model to differentiate between Cats and Dogs.

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, datasets
import time
%matplotlib inline

In [ ]:
#torch.__version__

In [ ]:
#import sys
#sys.version

**This file is intended to run on Google Colab**

Check if GPU is available, if not change the runtime

In [ ]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print('Using gpu: %s ' % torch.cuda.is_available())

## Download the Data

The data from [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) consists of two files: images.tar.gz and annotations.tar.gz

In [ ]:
#%pwd

In [ ]:
#%mkdir data
#%cd data

In [ ]:
#!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
#!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

Extract the downloaded files

In [ ]:
#!tar zxvf images.tar.gz
#!tar zxvf annotations.tar.gz

# Data Wrangling

In [ ]:
#%ls

In [ ]:
#ls annotations

In [ ]:
#!head annotations/test.txt

In [ ]:
#!head annotations/trainval.txt

In [ ]:
#%mkdir test trainval

In [ ]:
#%ls

In [ ]:
'''
def check_dir(dir_path):
    dir_path = dir_path.replace('//','/')
    os.makedirs(dir_path, exist_ok=True)'''

In [ ]:
'''
import re
import os
import shutil
#Load Regex
pat = re.compile(r'_\d')
pwd = os.getcwd()
#define data wrangler function to format file system in desired order
def data_wrangle(folder, lowercase):
    #iterate through test file for list images to save to test dir
    with open(f'./annotations/{folder}.txt') as fp:
        line = fp.readline()
        while line:
            f,_,_,_ = line.split(' ')
            res,_ = pat.split(f)
            line = fp.readline()
            path = os.path.join(pwd,f"{folder}/",res)
            src = os.path.join(pwd,"images/",f"{f}.jpg")
            if not lowercase and not res.islower():
                check_dir(path)
                print("path '%s' created" %path)
                path = os.path.join(pwd,f"{folder}/{res}",f"{f}.jpg")
                shutil.copy(src, path)
                print(f"file from {src} copied to {path}")
            elif lowercase and res.islower():
                check_dir(path)
                print("path '%s' created" %path)
                path = os.path.join(pwd,f"{folder}/{res}",f"{f}.jpg")
                shutil.copy(src, path)
                print(f"file from {src} copied to {path}")'''

In [ ]:
#data_wrangle("test",False)

In [ ]:
#data_wrangle("test",True)

In [ ]:
#data_wrangle("trainval",False)

In [ ]:
#data_wrangle("trainval",True)

# DO NOT RUN ABOVE CODE AGAIN
#
#

In [ ]:
%ls test

In [ ]:
%ls trainval

# Data Processing

In [ ]:
%ls data
%cd data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, datasets
import time
import os
data_dir = os.getcwd()

All images need to be the same size to work with pytorch

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg_format = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

In [ ]:
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), vgg_format)
        for x in ['train', 'test']}

In [ ]:
os.path.join(data_dir, 'train')

In [ ]:
dsets['train'].classes

In [ ]:
dsets['train'].class_to_idx

In [ ]:
dset_sizes = {x: len(dsets[x]) for x in ['train', 'test']}
dset_sizes

In [ ]:
dset_classes = dsets['train'].classes

In [ ]:
loader_train = torch.utils.data.DataLoader(dsets['train'], batch_size = 8, shuffle = True, num_workers=6)

In [ ]:
#?torch.utils.data.DataLoader

In [ ]:
loader_valid = torch.utils.data.DataLoader(dsets['test'], batch_size = 8, shuffle = True, num_workers=6)

Check DataLoader

In [ ]:
count = 1
for data in loader_valid:
    print(count, end=',')
    if count == 1:
        inputs_try,labels_try = data
    count += 1

In [ ]:
labels_try

In [ ]:
inputs_try.shape

Defining function to display images:

In [ ]:
def imshow(inp, title=None):
#   Imshow for Tensor.
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = np.clip(std * inp + mean, 0,1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
# Make a grid from batch
out = torchvision.utils.make_grid(inputs_try)

imshow(out, title=[dset_classes[x] for x in labels_try])

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(loader_train))

n_images=8

# Make a grid from batch
out = torchvision.utils.make_grid(inputs[0:n_images])

imshow(out, title=[dset_classes[x] for x in classes[0:n_images]])

In [ ]:
model_vgg = models.vgg16(weights='DEFAULT')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inputs_try , labels_try = inputs_try.to(device), labels_try.to(device)

model_vgg = model_vgg.to(device)

In [ ]:
outputs_try = model_vgg(inputs_try)

In [ ]:
outputs_try

In [ ]:
outputs_try.shape

In [ ]:
print(model_vgg)

In [ ]:
for param in model_vgg.parameters():
    param.requires_grad = False
    model_vgg.classifier._modules['6'] = nn.Linear(4096,37)
    model_vgg.classifier._modules['7'] = torch.nn.LogSoftmax(dim = 1)    

In [ ]:
print(model_vgg.classifier)

Load Model onto device(GPU,CPU,TPU)

In [ ]:
model_vgg = model_vgg.to(device)

# Training Fully Connected Models

## Creating loss function and optimizer

In [ ]:
criterion = nn.NLLLoss()
lr = 0.001
optimizer_vgg = torch.optim.SGD(model_vgg.classifier[6].parameters(),lr = lr)

## Train the Model

In [ ]:
def train_model(model,dataloader,size,epochs=1,optimizer=None):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        for inputs,classes in dataloader:
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            loss = criterion(outputs,classes)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))

In [ ]:
%%time
train_model(model_vgg,loader_train,size=dset_sizes['train'],epochs=2,optimizer=optimizer_vgg)

In [ ]:
def test_model(model,dataloader,size):
    model.eval()
    predictions = np.zeros(size)
    all_classes = np.zeros(size)
    all_proba = np.zeros((size,37))
    i = 0
    running_loss = 0.0
    running_corrects = 0
    #print(size)
    for inputs,classes in dataloader:
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        loss = criterion(outputs,classes)           
        _,preds = torch.max(outputs.data,1)
            # statistics
        running_loss += loss.data.item()
        running_corrects += torch.sum(preds == classes.data)
        predictions[i:i+len(classes)] = preds.to('cpu').numpy()
        all_classes[i:i+len(classes)] = classes.to('cpu').numpy()
        all_proba[i:i+len(classes),:] = outputs.data.to('cpu').numpy()
        i += len(classes)
    epoch_loss = running_loss / size
    epoch_acc = running_corrects.data.item() / size
    print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))
    return predictions, all_proba, all_classes

In [ ]:
predictions, all_proba, all_classes = test_model(model_vgg,loader_valid,size=dset_sizes['test'])

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(loader_valid))

out = torchvision.utils.make_grid(inputs[0:n_images])

imshow(out, title=[dset_classes[x] for x in classes[0:n_images]])

Compute the predictions made by your network for inputs[:n_images] and the associated probabilities.

Hint: use torch.max and torch.exp.

Do not forget to put your inputs on the device!

In [ ]:
outputs = model_vgg(inputs[:n_images].to(device))
print(torch.exp(outputs))

In [ ]:
vals_try, preds_try = torch.max(outputs,1)

In [ ]:
preds_try

In [ ]:
classes[:n_images]

In [ ]:
torch.exp(vals_try)

# Speeding up the learning process by precomputing features

In [ ]:
def preconvfeat(dataloader):
    conv_features = []
    labels_list = []
    for data in dataloader:
        inputs,labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features,labels_list)

In [ ]:
%%time
conv_feat_train,labels_train = preconvfeat(loader_train)

In [ ]:
conv_feat_train.shape

In [ ]:
%%time
conv_feat_valid,labels_valid = preconvfeat(loader_valid)

# Creating a new data generator
we will not load images anymore, so we need to build our own data loader.

In [ ]:
dtype=torch.float
datasetfeat_train = [[torch.from_numpy(f).type(dtype),torch.tensor(l).type(torch.long)] for (f,l) in zip(conv_feat_train,labels_train)]
datasetfeat_train = [(inputs.reshape(-1), classes) for [inputs,classes] in datasetfeat_train]
loaderfeat_train = torch.utils.data.DataLoader(datasetfeat_train, batch_size=128, shuffle=True)

now you can train for more epochs.

In [ ]:
%%time
train_model(model_vgg.classifier,dataloader=loaderfeat_train,size=dset_sizes['train'],epochs=80,optimizer=optimizer_vgg)

In [ ]:
datasetfeat_valid = [[torch.from_numpy(f).type(dtype),torch.tensor(l).type(torch.long)] for (f,l) in zip(conv_feat_valid,labels_valid)]
datasetfeat_valid = [(inputs.reshape(-1), classes) for [inputs,classes] in datasetfeat_valid]
loaderfeat_valid = torch.utils.data.DataLoader(datasetfeat_valid, batch_size=128, shuffle=False)

Now you can compute the accuracy on the test set.

In [ ]:
predictions, all_proba, all_classes = test_model(model_vgg.classifier,dataloader=loaderfeat_valid,size=dset_sizes['test'])

# Confusion matrix
For 37 classes, plotting a confusion matrix is useful to see the performance of the algorithm per class.

In [ ]:
!pip3 install -U scikit-learn

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
def make_fig_cm(cm):
    fig = plt.figure(figsize=(12,12))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    tick_marks = np.arange(37);
    plt.xticks(tick_marks, dset_classes, rotation=90);
    plt.yticks(tick_marks, dset_classes, rotation=0);
    plt.tight_layout();
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        coeff = f'{cm[i, j]}'
        plt.text(j, i, coeff, horizontalalignment="center", verticalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Actual');
    plt.xlabel('Predicted');

In [ ]:
cm = confusion_matrix(all_classes,predictions)


In [ ]:
make_fig_cm(cm)

Here, you see that american pit bull terrier are often predicted as staffordshire bull terrier but overall the algorithm gives pretty good results!

Now, I will take a resnet34 model to modify it

In [ ]:
import torchvision
model_resnet = torchvision.models.resnet34(weights='DEFAULT')

In [ ]:
print(model_resnet)

In [ ]:
print(model_resnet.fc)

In [ ]:
print(model_resnet)

replace the last layer to 1000 inputs and 37 outputs

In [ ]:
model_resnet.eval()
for param in model_resnet.parameters():
    param.requires_grad = False
# your code here
model_resnet.fc = nn.Linear(512, 37)

In [ ]:
print(model_resnet.fc)

Create a soft max layer

In [ ]:
model_resnet_lsm = nn.Sequential(model_resnet, torch.nn.LogSoftmax(dim = 1))

check everything is working

In [ ]:
inputs_try , labels_try = inputs_try.to(device), labels_try.to(device)
model_resnet_lsm = model_resnet_lsm.to(device)
outputs_try = model_resnet_lsm(inputs_try)

In [ ]:
outputs_try.shape

In [ ]:
print(model_resnet_lsm[0].fc)

In [ ]:
lr = 0.001
optimizer_resnet = torch.optim.SGD(model_resnet_lsm[0].fc.parameters(),lr = lr)

In [ ]:
%%time
train_model(model_resnet_lsm,loader_train,size=dset_sizes['train'],epochs=30,optimizer=optimizer_resnet)

In [ ]:
%%time
predictions, all_proba, all_classes = test_model(model_resnet_lsm,loader_valid,size=dset_sizes['test'])

In [ ]:
cm = confusion_matrix(all_classes,predictions)
make_fig_cm(cm)